In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
df = pd.read_csv('/content/news.csv', encoding='ISO-8859-1')

df['text'] = df['text'].str.replace(r'[^\x00-\x7F]+', '')

df.to_csv('/content/news.csv', index=False)

<ipython-input-3-a277be9a7702>:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/news.csv', encoding='ISO-8859-1')
<ipython-input-3-a277be9a7702>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r'[^\x00-\x7F]+', '')


In [4]:
print("Shape of the DataFrame:", df.shape)
print(df.head())

Shape of the DataFrame: (7795, 141)
  Unnamed: 0                                              title  \
0       8476                     You Can Smell Hillaryâs Fear   
1      10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2       3608        Kerry to go to Paris in gesture of sympathy   
3      10142  Bernie supporters on Twitter erupt in anger ag...   
4        875   The Battle of New York: Why This Primary Matters   

                                                text label Unnamed: 4  \
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE        NaN   
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE        NaN   
2  U.S. Secretary of State John F. Kerry said Mon...  REAL        NaN   
3   Kaydee King (@KaydeeKing) November 9, 2016 Th...  FAKE        NaN   
4  It's primary day in New York and front-runners...  REAL        NaN   

  Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 131  \
0        NaN        NaN        NaN        Na

In [5]:
df = pd.read_csv('/content/news.csv')

df = df.dropna(axis=1, how='all')

<ipython-input-5-0d8d08dc844b>:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/news.csv')


In [6]:
missing_data = df.isnull().sum()
print("Missing Data:\n", missing_data)

print("Shape of the DataFrame after cleaning:", df.shape)

Missing Data:
 Unnamed: 0       219
title            610
text             866
label           1040
Unnamed: 4      7477
                ... 
Unnamed: 136    7794
Unnamed: 137    7794
Unnamed: 138    7794
Unnamed: 139    7794
Unnamed: 140    7794
Length: 141, dtype: int64
Shape of the DataFrame after cleaning: (7795, 141)


In [7]:
numerical_columns = ['Unnamed: 0']
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())

text_columns = ['title', 'text']
df[text_columns] = df[text_columns].fillna('')

df['label'] = df['label'].fillna(df['label'].mode()[0])

missing_data = df.isnull().sum()
print("Missing Data after handling:\n", missing_data)

print("Shape of the DataFrame after handling missing values:", df.shape)

Missing Data after handling:
 Unnamed: 0       219
title              0
text               0
label              0
Unnamed: 4      7477
                ... 
Unnamed: 136    7794
Unnamed: 137    7794
Unnamed: 138    7794
Unnamed: 139    7794
Unnamed: 140    7794
Length: 141, dtype: int64
Shape of the DataFrame after handling missing values: (7795, 141)


<ipython-input-7-9660810ad7a3>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())


In [8]:
labels=df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.2, random_state=7)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)

In [11]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 76.46%


In [12]:

confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[578,  38],
       [240, 614]])